In [ ]:
import cPickle as pickle
import sys
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import HTML
import jinja2 as ja
from matplotlib import rcParams, rc, font_manager
from matplotlib.ticker import AutoMinorLocator
import json
from os.path import expanduser

from Chandra.Time import DateTime
from Ska.engarchive import fetch_eng as fetch

home = expanduser("~")
sys.path.append(home + '/AXAFLIB/pylimmon/')
import pylimmon

%matplotlib inline

In [ ]:
def check_violations(thermdict, t1, t2):
    """Check a list of MSIDs for limit/expected state violations.
    
    :param thermdict: Dictionary of MSID information (MSID name, condition type, etc.)
    :param t1: String containing start date in HOSC format
    :param t2: String containgin stop date in HOSC format
    
    """
    t1 = DateTime(t1).date
    t2 = DateTime(t2).date

    allviolations = {}
    missingmsids = []
    checkedmsids = []
    for key in thermdict.keys():
        greta_msid = thermdict[key]['greta_msid']
        try:
            if thermdict[key]['type'] == 'limit':
                if "wide" in greta_msid.lower():
                    violations = handle_widerange_cases(key, t1, t2, greta_msid)
                    checkedmsids.append(key)
                else:
                    violations = pylimmon.check_limit_msid(key, t1, t2, greta_msid=greta_msid)
                    checkedmsids.append(key)
            elif thermdict[key]['type'] == 'expst':
                violations = pylimmon.check_state_msid(key, t1, t2, greta_msid=greta_msid)
                checkedmsids.append(key)
                
            if len(violations) > 0:
                allviolations[key] = process_violations(key, violations)
            
        except IndexError:
            print('{} not in DB'.format(key))
            missingmsids.append(key)

    return allviolations, missingmsids, checkedmsids

def handle_widerange_cases(key, t1, t2, greta_msid):
    """Handle special widerange MSIDs.
    
    :param key: Name of MSID as represented in Ska Engineering Archive
    :param t1: String containing start time in HOSC format
    :param t2: String containgin stop time in HOSC format
    :greta_msid: Name of MSID as represented in GRETA
    
    Note: Some MSID names differ between Ska and GRETA. Widerange MSIDs are one such case. For example
    OOBTHR35 is used for this measurement in both Ska and GRETA before this MSID was switched to 
    widerange read mode. Afterwards GRETA uses OOBTHR35_WIDE whereas Ska still uses OOBTHR35 for 
    continuity.
    """
    if DateTime(t2).secs <= DateTime('2014:342:16:30:00').secs:
        violations = pylimmon.check_limit_msid(key, t1, t2, greta_msid=key)
    elif DateTime(t1).secs >= DateTime('2014:342:16:33:00').secs:
        violations = pylimmon.check_limit_msid(key, t1, t2, greta_msid=greta_msid)
    else:
        t2_a = np.min((DateTime(t2).secs, DateTime('2014:342:16:30:00').secs))
        violations = pylimmon.check_limit_msid(key, t1, t2_a, greta_msid=key)
        t1_b = np.min((DateTime(t2).secs, DateTime('2014:342:16:33:00').secs))
        violations_b = pylimmon.check_limit_msid(key, t1_b, t2, greta_msid=greta_msid)

        violations.extend(violations_b)
        
    return violations

def process_violations(msid, violations):
    """Add contextual information for any limit/expected state violations.
    
    :param msid: Current mnemonic
    :param violations: List of individual violations (list of tuples)
    
    """
    data = fetch.Msid(msid, violations[0][0][0], violations[0][0][-1], stat='5min')
    try:
        desc = data.tdb.technical_name
    except:
        desc = 'No Description in TDB'
        
    violation_dict = {}
    for v in violations:
        limtype = v[-1]
        if 'high' in limtype.lower():
            if limtype not in violation_dict.keys():
                violation_dict.update({limtype:{'starttime':v[0][0], 'stoptime':v[0][-1], 'num_excursions':1,
                                                'extrema':np.max(v[1]), 'limit':v[2][0], 'setid':v[3][0]}})
            else:
                violation_dict[limtype]['extrema'] = np.max((np.max(v[1]), violation_dict[limtype]['extrema']))
                violation_dict[limtype]['starttime'] = np.min((v[0][0], violation_dict[limtype]['starttime']))
                violation_dict[limtype]['stoptime'] = np.max((v[0][0], violation_dict[limtype]['stoptime']))
                violation_dict[limtype]['num_excursions'] = violation_dict[limtype]['num_excursions'] + 1
                
        elif 'low' in limtype.lower():
            if limtype not in violation_dict.keys():
                violation_dict.update({limtype:{'starttime':v[0][0], 'stoptime':v[0][-1], 'num_excursions':1,
                                                'extrema':np.min(v[1]), 'limit':v[2][0], 'setid':v[3][0]}})
            else:
                violation_dict[limtype]['extrema'] = np.min((np.min(v[1]), violation_dict[limtype]['extrema']))
                violation_dict[limtype]['starttime'] = np.min((v[0][0], violation_dict[limtype]['starttime']))
                violation_dict[limtype]['stoptime'] = np.max((v[0][0], violation_dict[limtype]['stoptime']))
                violation_dict[limtype]['num_excursions'] = violation_dict[limtype]['num_excursions'] + 1

        elif 'state' in limtype.lower():
            if limtype not in violation_dict.keys():
                violation_dict.update({limtype:{'starttime':v[0][0], 'stoptime':v[0][-1], 'num_excursions':1,
                                                'extrema':v[1][0], 'limit':v[2][0], 'setid':v[3][0]}})
            else:
                violation_dict[limtype]['starttime'] = np.min((v[0][0], violation_dict[limtype]['starttime']))
                violation_dict[limtype]['stoptime'] = np.max((v[0][0], violation_dict[limtype]['stoptime']))
                violation_dict[limtype]['num_excursions'] = violation_dict[limtype]['num_excursions'] + 1

                
    for limittype in ['warning_low', 'caution_low', 'caution_high', 'warning_high', 'state']:
        if limittype in violation_dict.keys():
            violation_dict[limittype]['duration'] = (violation_dict[limittype]['stoptime'] 
                - violation_dict[limittype]['starttime']) / 3600.
            violation_dict[limittype]['description'] = desc
            violation_dict[limittype]['startdate'] = DateTime(violation_dict[limittype]['starttime']).date
            violation_dict[limittype]['stopdate'] = DateTime(violation_dict[limittype]['stoptime']).date
            
    return violation_dict

In [ ]:
thermdict, missing, notinarchive = pickle.load(open(home + '/AXAFDATA/weekly_report_data/thermalmsiddata.pkl','r'))

t1 = DateTime('2015:265:20:00:00.000').date
t2 = DateTime('2015:266:00:00:00.000').date
dayrange = (t1[:9] + '-' + t2[:9]).replace(':', '')

allviolations, missingmsids, checkedmsids = check_violations(thermdict, t1, t2)

In [ ]:
hours = (DateTime(t2).secs - DateTime(t1).secs)/3600.
print('\nChecked {} MSIDs at full resolution over {} hour period, {} MSIDs left unchecked (missing)\n'.format(
        len(checkedmsids), hours, len(missingmsids)) )

env = ja.Environment(loader=ja.FileSystemLoader('./templates'))
template = env.get_template('index.htm')
webpage = template.render(dayrange=dayrange, violations=allviolations)
HTML(webpage)